In [9]:
import requests
import logging
import time
from mechanize.polyglot import getitem
import js2py
import mechanize
from threading import Thread
import csv

#self.new_get_auth()
#self.new_get_info()
#self.filter_auth()
#self.find_email()
#self.save_output()

logger = logging.getLogger(__name__)

class get_auth_id:
    def __init__(self,query):
        self.query = query
        self.authors = {}
        self.apisList = self.get_apis()
        #self.get_all_pages_result()
        
        pass

    def get_apis(self):
        #init = 'bfd55d016ed308a6de2e9d39e7139dec'
        init = '8579fef278e4149355865b4bcdaed8f9'
        keys_list = open("keys.txt", 'r').readlines()
        self.API_KEY = init
        return keys_list

    def give_new_key(self):
        from random import choice
        self.API_KEY = choice(self.apisList).replace('\n','')

    def get_single_page(self,url,param={}):
        header = {'Accept':'application/json','X-ELS-APIKey': self.API_KEY}
        loop_counter = 0
        while (True):
            r = requests.get(url,headers=header,params=param)
            if(r.status_code == 200 ):break
            if(r.status_code == 429 ): self.give_new_key()
            loop_counter +=1
            if(loop_counter == 10):
                print("new error has arrived")
                print (r)
                print (r.json())
        return r.json()['search-results']
    
    def get_first_search_page_link(self):
        url = "https://api.elsevier.com/content/search/scopus?"
        #parameter = {'query' : "SRCTITLE("+self.query+")",'date':str(self.stime)+'-'+str(self.ftime),'cursor' : '*','view' : 'COMPLETE'}
        #parameter = {'query' : "SRCTITLE("+self.query+")",'date':str(self.stime)+'-'+str(self.ftime),'view' : 'STANDARD'}
        parameter = {'query' : "SRCTITLE("+self.query+")  AND ( LIMIT-TO ( DOCTYPE , \"cp\" ) )",'cursor' : '*','view' : 'COMPLETE'} # removed date

        # SRCTITLE ( "2022 4th International Conference on Blockchain Computing and Applications, BCCA 2022" ) AND ( LIMIT-TO ( DOCTYPE , "cp" ) )

        links = self.get_single_page(url,param=parameter)['link']
        for item in links:
            if (item["@ref"] == "first"):
                return item['@href']
        
    def get_authors_id(self,page):
        entry = page['entry']

        for item in entry:
            if(not('author-count' in item and '$' in item['author-count'] ) or item['author-count']['$']=='0'):
                continue
            for i in range(int(item['author-count']['$'])):
                try:
                    aut = item['author'][i]['authid']
                    self.authors[aut]=[]
                except:continue
        pass

    def get_next_link(self,page):
        for item in page['link']:
            if (item["@ref"] == "next"):
                return item['@href']

    def is_last(self,current_link,page):
        return (current_link == self.get_next_link(page) )

    def get_all_pages_result(self):
        current_link = self.get_first_search_page_link()
        current_page = self.get_single_page(current_link)
        self.page = 1
        while not self.is_last(current_link,current_page):
            self.get_authors_id(current_page)
            current_link = self.get_next_link(current_page)
            if not current_link == None:
                current_page = self.get_single_page(current_link)
                
            print('next')
            self.page+=1

class get_info():
    def __init__(self,authors):
        self.authors = authors
        self.apisList = self.get_apis()
        self.get_all_authors_info()


    def get_apis(self):
        #init = 'bfd55d016ed308a6de2e9d39e7139dec'
        init = '8579fef278e4149355865b4bcdaed8f9'
        keys_list = open("keys.txt", 'r').readlines()
        self.API_KEY = init
        return keys_list

    def give_new_key(self):
        from random import choice
        self.API_KEY = choice(self.apisList).replace('\n','')

    def load_single_author_info(self,authid,j):
        loop_counter = 0
        url = "https://api.elsevier.com/content/author/author_id/"
        header = {'Accept':'application/json','X-ELS-APIKey': self.API_KEY}
        while(True):
            resp = requests.get( url+authid, headers = header, params = {'view':'ENHANCED'} )
            if(resp.status_code == 200 ):break
            if(resp.status_code == 429 and j == 1): self.give_new_key()
            if(resp.status_code == 429):continue
            if(resp.status_code == 404):break
            
            loop_counter +=1
            if(loop_counter > 0):
                print("new error has arrived" + authid)
                print (resp)
                print (resp.json())
                break
        if(resp.status_code == 404):return
        if(not isinstance(resp.json()['author-retrieval-response'], list)):return
        return resp.json()
    
    def get_singe_author_info(self,authid,j):
        j = self.load_single_author_info(authid,j)

        surname = j["author-retrieval-response"][0]['author-profile']['preferred-name']['surname']
        givenName = j["author-retrieval-response"][0]['author-profile']['preferred-name']['given-name']
        documents = j["author-retrieval-response"][0]["coredata"]["document-count"]
        citation = j["author-retrieval-response"][0]["coredata"]["citation-count"]
        HIndex = j["author-retrieval-response"][0]["h-index"]
        if 'affiliation-current' not in j['author-retrieval-response'][0]['author-profile'] :
            affiliation = "NotGIVEN"
        else:
            if(not isinstance(j['author-retrieval-response'][0]['author-profile']['affiliation-current']['affiliation'], list)):
                affiliation = j['author-retrieval-response'][0]['author-profile']['affiliation-current']['affiliation']['ip-doc']
            else:
                affiliation = j['author-retrieval-response'][0]['author-profile']['affiliation-current']['affiliation'][0]['ip-doc']
        if('afdispname' not in affiliation):
            affiliation = "NotGIVEN"
            country = "NotGIVEN"
        else:
            if('address' not in affiliation or affiliation['address']==None or'country' not in affiliation['address']):
                country = "NotGIVEN"
            else:
                country = affiliation['address']['country']
            affiliation=affiliation['afdispname']

        if(givenName==None):givenName = "NotGIVEN"
        if(surname==None):surname = "NotGIVEN"
        if(documents == None):documents = "0"
        if(citation == None):citation = "0"
        if(HIndex == None):HIndex = "0"
        self.authors[authid] = {'name':givenName,
                'surname':surname,
                'documents':int(documents),
                'citation':int(citation),
                'hindex':int(HIndex),
                'affilliation':affiliation,
                'country':country}
    
    def get_all_authors_info(self):
        NUM = 40
        s = list(self.authors.keys())
        thread=[]
        if(NUM>len(s)):NUM = len(s)
        for i in range(len(s)):
            #s[i] = s[i][0:len(s[i])-1]
            print(s[i])
            if(i<NUM):
                thread.append(Thread(target = self.get_singe_author_info,args =(s[i],i+1)))
                thread[i].start()
                continue
            f = False
            while(not f):
                for j in range(NUM):
                    if(not thread[j].is_alive()):
                        f = True
                        thread[j]=Thread(target = self.get_singe_author_info,args =(s[i],j+1))
                        thread[j].start()
                        break
        for i in range(NUM):
            thread[i].join()

class filter_authors():
    def __init__(self, authors):
        self.authors = authors
        self.apisList = self.get_apis()
        print(self.authors)

        pass

    def filterHindex(self,minH):
        self.authors = dict(filter(lambda x : x[1]['hindex']>= minH,self.authors.items()))
        #print(self.authors)
        pass

    def get_apis(self):
        #init = 'bfd55d016ed308a6de2e9d39e7139dec'
        init = '8579fef278e4149355865b4bcdaed8f9'
        keys_list = open("keys.txt", 'r').readlines()
        self.API_KEY = init
        return keys_list

    def give_new_key(self):
        from random import choice
        self.API_KEY = choice(self.apisList).replace('\n','')

    def get_single_author_age(self,authid,j):
        loop_counter = 0
        url = "http://api.elsevier.com/content/search/scopus?"
        header = {'Accept':'application/json','X-ELS-APIKey': self.API_KEY}
        param = {'query' : 'AU-ID('+authid+')','date' : self.stime+'-'+self.ftime}
        while(True):
            resp = requests.get( url+authid, headers = header, params = param )
            if(resp.status_code == 200 ):break
            if(resp.status_code == 429 and j == 1): self.give_new_key()
            if(resp.status_code == 429):continue
            if(resp.status_code == 404):break
            
            loop_counter +=1
            if(loop_counter > 0):
                print("new error has arrived" + authid)
                print (resp)
                print (resp.json())
                break
        if(resp.status_code == 404):return
        self.authors[authid]['inperiod'] =  int(resp.json()['search-results']['opensearch:totalResults'])
    
    def get_all_authors_age(self):
        NUM = 40
        s = list(self.authors.keys())
        thread=[]
        if(NUM>len(s)):NUM = len(s)
        for i in range(len(s)):
            #s[i] = s[i][0:len(s[i])-1]
            print(s[i])
            if(i<NUM):
                thread.append(Thread(target = self.get_single_author_age,args =(s[i],i+1)))
                thread[i].start()
                continue
            f = False
            while(not f):
                for j in range(NUM):
                    if(not thread[j].is_alive()):
                        f = True
                        thread[j]=Thread(target = self.get_single_author_age,args =(s[i],j+1))
                        thread[j].start()
                        break
        for i in range(NUM):
            thread[i].join()

    def filterAge (self):
        self.get_all_authors_age()
        #print(self.authors)
        self.authors = dict(filter(lambda x : x[1]['documents'] == x[1]['inperiod'] ,self.authors.items()))
        print(self.authors)
        pass

class get_email():
    def __init__(self,authors):
        self.authors = authors
        self.apisList = self.get_apis()
        self.get_all_authors_email()
        pass

    def get_apis(self):
        #init = 'bfd55d016ed308a6de2e9d39e7139dec'
        init = '8579fef278e4149355865b4bcdaed8f9'
        keys_list = open("keys.txt", 'r').readlines()
        self.API_KEY = init
        return keys_list

    def give_new_key(self):
        from random import choice
        self.API_KEY = choice(self.apisList).replace('\n','')

    def get_page(self,authid,j,start):
        loop_counter = 0
        url ="http://api.elsevier.com/content/search/scopus?"
        header = {'Accept':'application/json','X-ELS-APIKey': self.API_KEY}
        param = {'start':str(start),'query' : 'AU-ID('+str(authid)+')','count' : '50','sort':'-coverDate'}
        while(True):
            resp = requests.get( url+authid, headers = header, params = param )
            if(resp.status_code == 200 ):break
            if(resp.status_code == 429 and j == 1): self.give_new_key()
            if(resp.status_code == 429):continue
            if(resp.status_code == 404):break
            
            loop_counter +=1
            if(loop_counter > 0):
                print("new error has arrived" + authid)
                print (resp)
                print (resp.json())
                break
        return resp.json()

    def find_paper_urls(self,authid,j):
        start = int(self.papercount/50)*50
        page = self.get_page(authid,j,start)
        if page['search-results']['opensearch:totalResults'] == self.papercount+1: self.signal=2
        return list(map(lambda x : x['link'][2]['@href'],page['search-results']['entry'][:]))   

        
    def find_coded_string(self,document):
        l = "/cdn-cgi/l/email-protection#"
        index = 0 
        if l in document: self.signal=1
        else: 
            return
        index = document.index(l)
        document[index:].index('"')
        href = document[document.index(l):document.index(l)+document[index:].index('"')]
        href = 'file://'+ href
        return href   

    def decode_email(self,coded):
        if coded == None: return 'Null'
        r = js2py.eval_js('''    function r(e, t) {
        var r = e.substr(t, 2);
        return parseInt(r, 16)
            }''')
        n = js2py.eval_js('''    function n(n, c,r) {
                for (var o = "", a = r(n, c), i = c + 2; i < n.length; i += 2) {
                    var l = r(n, i) ^ a;
                    o += String.fromCharCode(l)
                }
                try {
                    o = decodeURIComponent(escape(o))

                } catch (u) {
                    console.log('ddd')
                }
                return o
            }''')
        email = n(coded,35,r) 
        print(email)
        return email

    def find_email(self,authid,j):
        b = mechanize.Browser()
        b.set_handle_robots(False)
        b._factory.is_html = True

        b.addheaders = [('User-agent',
                        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/45.0.2454101'
                        )]

        self.papercount = 0
        self.signal = 0
        while not self.signal==2:
            urls = self.find_paper_urls(authid,j)
            for url in urls:
                res = b.open(url)
                coded = self.find_coded_string(res.read().decode())
                if coded ==None : self.signal=0
                if self.signal ==1 and coded!='':
                    email = self.decode_email(coded)
                    self.authors[authid]['email'] = email
                    return
                if self.signal == 2:
                    self.authors[authid]['email'] = 'Null'
                self.papercount+=1

        
    def get_all_authors_email(self):
        NUM = 40
        s = list(self.authors.keys())
        thread=[]
        if(NUM>len(s)):NUM = len(s)
        for i in range(len(s)):
            #s[i] = s[i][0:len(s[i])-1]
            print(s[i])
            if(i<NUM):
                thread.append(Thread(target = self.find_email,args =(s[i],i+1)))
                thread[i].start()
                continue
            f = False
            while(not f):
                for j in range(NUM):
                    if(not thread[j].is_alive()):
                        f = True
                        thread[j]=Thread(target = self.find_email,args =(s[i],j+1))
                        thread[j].start()
                        break
        for i in range(NUM):
            thread[i].join()

class savecsv():
    def __init__(self,authors,fname,lname,email,aff,country,doc,cit,hind,id,output_name):
        self.authors = authors
        self.fname = fname
        self.lname = lname
        self.email = email
        self.aff = aff
        self.country = country
        self.doc = doc
        self.cit = cit
        self.hind = hind
        self.id = id
        self.output_name = output_name
        self.save()
    
    def save(self):
        for i in self.authors:
            self.authors[i]['id'] = str(i)

        header = ['id']
        if self.fname : header.append('name')
        if self.lname : header.append('surname')
        if self.aff : header.append('affilliation')
        if self.country : header.append('country')
        if self.doc : header.append('documents') 
        if self.cit : header.append('citation') 
        if self.hind : header.append('hindex') 
        if self.email : header.append('email') 

        with open (self.output_name,'w',newline='',encoding = 'utf-8') as f:
            w = csv.DictWriter(f,fieldnames = header, extrasaction='ignore')
            w.writeheader()
            w.writerows(list(self.authors.values()))
            f.close()

class getAllData():

    def new_get_auth(self):
        #pre
        logger.error('Looking for all Author ids')
        logger.error('this might take up to 30-40 seconds')
        logger.error('Please Wait ...')
        t1 = time.time()
        #main
        self.authors={}
        query = "2022 4th International Conference on Blockchain Computing and Applications, BCCA 2022"
        self.extractor = get_auth_id(query)
        self.extractor.get_all_pages_result()
        self.authors = self.extractor.authors

        #post
        t2 = time.time()
        logger.error(f'extracted {len (self.extractor.authors.keys())} Author ids in {int(t2 - t1)} seconds')

    def new_get_info(self):
        logger.error('trying to get some informations about theese authors...')
        logger.error('this might take up to 1-2 minutes')
        logger.error('please wait ...')
        t1 = time.time()
        #ui.new_get_info()
        self.info_class = get_info(self.authors)
        #self.info_class = get_info(self.extractor.authors)
        t2 = time.time()
        logger.error(f'got {len (self.info_class.authors.keys())} Author information in {int(t2- t1)} seconds')
        #ui.Signal=3
        print('finished')
        #logger.error(str(ui.info_class.authors))

    def filter_auth(self):
        logger.error('trying to filter authors according to your inputs \n this shoud not take too long')
        logger.error('please Wait ...')
        t1 = time.time()
        #ui.filter_auth()
        self.filterd_class = filter_authors(self.info_class.authors)
        t2 = time.time()
        time.sleep(0.3)
        logger.error(f'{len(self.filterd_class.authors.keys())} authors left \n proccess took {int(t2-t1)} seconds')
        pass

    def find_email(self):
        logger.error('trying to find remaining authors email')
        logger.error('please Wait ...')
        t1 = time.time()
        #ui.find_email()
        self.email_class = get_email(self.filterd_class.authors)
        t2 = time.time()
        time.sleep(0.3)
        logger.error(f'{len(self.email_class.authors.keys())} authors email found \n proccess took {int(t2-t1)} seconds')
        #logger.error(str(ui.email_class.authors))
        pass

    def save_output(self):
        fname = True
        lname = True
        email = True
        aff = True
        country = True
        doc = True
        cit = True
        hind = True
        id = True
        output_name = "output.csv"
        logger.error('fetching data done.\ntrying to save outputs')
        #ui.save_output()
        savecsv(self.email_class.authors,fname,lname,email,aff,country,doc,cit,hind,id,output_name)
        logger.error('finish')
        logger.error('**************')
        print('finish')
        pass

In [10]:
c = getAllData()
c.new_get_auth()
#self.new_get_info()
#self.filter_auth()
#self.find_email()
#self.save_output()

Looking for all Author ids
this might take up to 30-40 seconds
Please Wait ...


next


extracted 136 Author ids in 2 seconds


next


In [11]:
c.new_get_info()

trying to get some informations about theese authors...
this might take up to 1-2 minutes
please wait ...


57387495000
57963134000
57963134600
57962084700
35182858700
24473625800
57224819057
57203202954
6602076736
57221284581
36467092300
6602505210
57170217300
57225941692
7202528532
57210551520
55336363400
9275987500
57968393200
58373643800
57968704300
49862114100
57198028780
7004750176
57742123900
57212628517
15019127200
56398590900
56770825600
57967930500
57193613501
24921304700
57202044639
57219172919
6602410612
58842548000
57195626687
57250734300
57219176853
57968234500
57968704400
6602731292
57210223297
57194349729
35617663600
9042909100
14520494400
56045308100
8843325700
57219421852
7003797040
57968234400
55141650200
57968393100
55787681200
57220073777
57968540900
57967930600
55822409000
57967776900
57196850775
35173141900
57202377287
57203912525
57194441894
7202111773
23492776200
57968081100
56763278500
57216582729
36010503100
55665563600
57274971500
58267474200
57968849600
23096391900
57188647874
56418472900
27567940500
24824563500
56027475000
35226211500
35569999500
56847631300
660

got 136 Author information in 14 seconds


finished


In [12]:
c.filter_auth()

trying to filter authors according to your inputs 
 this shoud not take too long
please Wait ...


{'57387495000': {'name': 'Mohamed', 'surname': 'Ghanem', 'documents': 2, 'citation': 3, 'hindex': 1, 'affilliation': 'School of Sciences and Engineering', 'country': 'Egypt'}, '57963134000': {'name': 'Fadi', 'surname': 'Dawoud', 'documents': 2, 'citation': 3, 'hindex': 1, 'affilliation': 'School of Sciences and Engineering', 'country': 'Egypt'}, '57963134600': {'name': 'Habiba', 'surname': 'Gamal', 'documents': 2, 'citation': 3, 'hindex': 1, 'affilliation': 'School of Sciences and Engineering', 'country': 'Egypt'}, '57962084700': {'name': 'Eslam', 'surname': 'Soliman', 'documents': 2, 'citation': 3, 'hindex': 1, 'affilliation': 'School of Sciences and Engineering', 'country': 'Egypt'}, '35182858700': {'name': 'Tamer', 'surname': 'El-Batt', 'documents': 7, 'citation': 99, 'hindex': 4, 'affilliation': 'American University in Cairo', 'country': 'Egypt'}, '24473625800': {'name': 'Hossam S.', 'surname': 'Sharara', 'documents': 21, 'citation': 202, 'hindex': 7, 'affilliation': 'American Univ

136 authors left 
 proccess took 0 seconds


In [13]:
c.find_email()

trying to find remaining authors email
please Wait ...


57387495000
57963134000
57963134600
57962084700
35182858700
24473625800
57224819057
57203202954
6602076736
57221284581
36467092300
6602505210
57170217300
57225941692
7202528532
57210551520
55336363400
9275987500
57968393200
58373643800
57968704300
49862114100
57198028780
7004750176
57742123900
57212628517
15019127200
56398590900
56770825600
57967930500
57193613501
24921304700
57202044639
57219172919
6602410612
58842548000
57195626687
57250734300
57219176853
57968234500
57968704400
ozturaca@boun.edu.tr
gulshan3971@gmail.comyoussif.abuzied@aucegypt.edu

youssif.abuzied@aucegypt.edu


KeyboardInterrupt: 

yahya.shahsavari.1@ens.etsmtl.ca
balikakli@gmail.comricardo.j.bessa@inesctec.pt

jolly.parikh@bharatividyapeeth.edu
rsahaaot@gmail.com
haejoonjung@khu.ac.kr
youssif.abuzied@aucegypt.eduyoussif.abuzied@aucegypt.edu

bmsilva@fe.up.pt
youssif.abuzied@aucegypt.edu
gulshan3971@gmail.com
vikasmr@cse.iitk.ac.in
moayad.aloqaily@mbzuai.ac.ae
serena.nicolazzo@unimi.it
amna.basharat@nu.edu.pk


In [6]:
c.save_output()

fetching data done.
trying to save outputs
finish
**************


finish
